# Exploración y Análisis de Taquerías en la Ciudad de México utilizando Python y Ciencia de Datos

##  Introducción:
La Ciudad de México es mundialmente reconocida por su vasta y diversa oferta gastronómica, siendo las taquerías uno de los pilares fundamentales de la cultura culinaria urbana. Con miles de establecimientos repartidos por toda la capital, identificar patrones de ubicación, popularidad y accesibilidad puede aportar valor tanto para consumidores como para futuros emprendedores gastronómicos.

Este proyecto busca aplicar herramientas de programación en Python para explorar, visualizar y analizar datos reales de taquerías en CDMX, abarcando desde estadísticas descriptivas básicas hasta análisis espaciales y rankings ponderados.

El proposito principal de este análisis es encontrar los mejores tacos de la ciudad de Mexico quisimos llevar nuestro proyecto a otro nivel, ya que nos preguntamos si existía la posibilidad  de encontrar los datos de el API de google maps, ya que de los datos que las personas y los negocios proporcionan dáa a día a la plataforma de google maps son demasiado valiosos.

En CDMX y área conurbada existen más de 10000 taquerias las cueales se dividen entre locales y negocios callejeros, así que logramos extraer una muestra de 3000 mil datos de los cuales nos surguieron las siguientes preguntas: 

##### -¿Cuáles son los mejores tacos de la CDMX? 
##### -¿Cuáles son los mejores tacos cerca de ESCOM? 
##### -¿Cáles son los más pupulares? 
##### -¿Cuáles son los más recomendados? 
#####  Mejores tacos por delegación
##### ¿Cuáles son los peores tacos de CDMX?


## Objetivos 

### Objetivo General

 Analizar, visualizar y evaluar datos de taquerías en la Ciudad de México para identificar tendencias de popularidad, ubicación y accesibilidad utilizando herramientas de ciencia de datos.

### Objetivos específicos
1. Aplicar estadística descriptiva al data frame obtenido de las taquerías que están registradas en google maps, utilizando biblotecas de python como numpy y pandas
2. Encontrar taquerías destacadas con la información del DataFrame 
3. Encontrar la taquería con mejor calificación en toda la CDMX 
4. Encontrar la mejor taquería cercana a ESCOM
5. Destacas las taquerías con peores calificaciones
6. Realizar limpieza del DataFrame; y con este nuevo DataFrame aplicar estadistica descriptiva 
7. Agrupar y visualizar datos del DataFrame
8. Diseñar e implementar un ranking ponderado más justo para determinar las taquerías más recomendables 

## Metodología 
El proyecto se desarrolla en varias etapas, desde la obtención de datos hasta el análisis geográfico y ponderado

#### Primera parte: Obtención del DataFrame
El dataset fue generado utilizando la Google Maps Platform, usamos Places API, mediante consultas con palabras clave como "tacos" o "Taquerias", lo cual permitió recuperar información de establecimientos comerciales clasificados como taquerías en la capital.

El formato general de la solicitud HTTPS fue el siguiente:

```
response = requests.post(
    "https://places.googleapis.com/v1/places:searchText", // endpoint de Google
    headers=headers,
    data=json.dumps(payload)
)
```
Donde los headers preguntan por datos de Places API como numero de calificaciones de las taquerias, promedio de calificaciones, direccion, etc
Y el Payload es el string a buscar
```
payload= "textQuery" : "tacos"
```
Foto cordenadas

La extracción de datos se realizó mediante peticiones del tipo Text Search en la API, obteniendo los siguientes atributos por cada establecimiento:

name: nombre del lugar

address: dirección textual

lat, lng: coordenadas geográficas

rating: calificación promedio otorgada por los usuarios

userRatingCount: número de reseñas

priceLevel: nivel de precio categórico

website: sitio web del establecimiento (si está disponible)

La información fue almacenada en un archivo.csv y cargada en Python para su análisis posterior con pandas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import folium

# Leer el archivo limpio y ya ordenado
df = pd.read_csv("tacos_CDMX_sorted.csv")
print('Columnas disponibles: ',df.columns.tolist())


ModuleNotFoundError: No module named 'pandas'

### Mostramos todas las columnas de el dataset.
Columnas disponibles:  ['name', 'address', 'lat', 'lng', 'rating', 'userRatingCount', 'priceLevel', 'website']


### Mostramos el dataset completo 

In [ ]:
df

NameError: name 'df' is not defined

## Metodología 

### Segunda parte: Análisis Estadístico Descriptivo

#### Filtraremos taquerías con datos utiles
Quitaremos las que no tengan calificación y después las que tengan menos de 10 calificaciones.

In [ ]:
#UserRating count es el numero de calificaciones
dataFiltrada= df[df['userRatingCount']>=20]
print(dataFiltrada.shape)
dataFiltrada

# Como primer paso arrojaremos las taquerías mejor calificadas en la ciudad 


In [ ]:
top= dataFiltrada.sort_values(by='rating',ascending=False).head(15)
top
#sort_values ordena de mayor a menor el rating


In [ ]:
taqueriasExcelentes= dataFiltrada[dataFiltrada['rating']>=5]
taqueriasExcelentes= taqueriasExcelentes.sort_values(by='userRatingCount', ascending=False).head(300)
print(taqueriasExcelentes.shape)
taqueriasExcelentes
#Aqui podemos visualizzar las taquerías mejor calificadas y con un gran numero de reviews

In [ ]:
#con value_counts()contamos el numero de calificaciones por rating
# y lo ordenamos con sort_index()
# Esto nos da una idea de la distribucion de las calificaciones
numero_calificaciones= dataFiltrada['rating'].value_counts().sort_index()
numero_calificaciones


In [ ]:
# Graficar la distribución de calificaciones
plt.figure(figsize=(12, 5))
plt.hist(dataFiltrada['rating'], bins=np.arange(1, 5.1, 0.1), edgecolor='black')
plt.title('Distribución de calificaciones de taquerías')
plt.xlabel('Rating')
plt.ylabel('Cantidad de taquerías')
plt.grid(axis='y')
plt.show()

##### Promedio de calificaciones
Primero analizamos el promedio del taset completo.
Después el promedio de el daataset ya filtrados, sin las taquerías que tienen menos de 5 calificaciones.


In [ ]:
print('Media  general de calificaciones: ', df['rating'].mean())
print('Media  general de calificaciones filtradas: ', dataFiltrada['rating'].mean())

####### Con el promedio general de calificaciones es de 4.43 nos damos cuenta que la mayoría de los tacos con más calificaciones estan por debajo del promedio, por lo tanto nos podemos tomar el numero de calificaciones como la metrica para conseguir el mejor taco de la cuidad. 

####


#### Mostraremos las taquerías y las calificaciones pero en lugar de una tabla las muestra con un punto.


In [ ]:

# Extraer los datos
X = dataFiltrada['userRatingCount'].values
y = dataFiltrada['rating'].values

# Calcular la pendiente (m) y el intercepto (b) de la recta: y = mX + b
m, b = np.polyfit(X, y, 1)  # Regresión lineal de grado 1

# Predicción
y_pred = m * X + b

# Imprimir coeficientes
print("Coeficiente (pendiente):", m)
print("Intercepto:", b)

# Gráfica
plt.scatter(X, y, alpha=0.4, label='Datos reales')
plt.plot(X, y_pred, color='red', label='Regresión lineal')
plt.xlabel('Número de calificaciones')
plt.ylabel('Rating')
plt.title('Regresión lineal: Rating vs Número de calificaciones')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df

### Mejores taquerías por alcaldía 



In [ ]:
#Lista de Alcaldias 
alcaldias_cdmx = [
    'Álvaro Obregón', 'Azcapotzalco', 'Benito Juárez', 'Coyoacán', 'Cuajimalpa',
    'Cuauhtémoc', 'Gustavo A. Madero', 'Iztacalco', 'Iztapalapa', 'La Magdalena Contreras',
    'Miguel Hidalgo', 'Milpa Alta', 'Tláhuac', 'Tlalpan', 'Venustiano Carranza', 'Xochimilco'
]

#Función para extraer alcaldías 
def extraer_alcaldia(direccion):
    for alcaldia in alcaldias_cdmx:
        if alcaldia.lower() in direccion.lower():
            return alcaldia
    return "No identificada"

df_copia2 = df.copy()
df_filtrado2 = df_copia2[df_copia2['userRatingCount'] > 20]
df_filtrado2['alcaldia'] = df_filtrado2['address'].apply(extraer_alcaldia)

df_filtrado2

In [ ]:
# Obtener mejor taquería por alcaldía 
mejores_por_alcaldia_rating = df_filtrado2.loc[
    df_filtrado2.groupby('alcaldia')['rating'].idxmax()
].reset_index(drop=True)

mejores_por_alcaldia_rating



In [ ]:
mejores_por_alcaldia = pd.DataFrame()  # 1. Creamos un DataFrame vacío para guardar resultados

for alcaldia, grupo in df_filtrado2.groupby('alcaldia'):  # 2. Agrupamos el DataFrame por alcaldía y recorremos cada grupo
    max_rating = grupo['rating'].max()  # 3. Encontramos la calificación máxima en esa alcaldía
    mejores = grupo[grupo['rating'] == max_rating]  # 4. Filtramos las taquerías con esa calificación máxima
    mejor = mejores.loc[mejores['userRatingCount'].idxmax()]  # 5. De las mejores, elegimos la que tiene más reseñas
    mejores_por_alcaldia = pd.concat([mejores_por_alcaldia, mejor.to_frame().T], ignore_index=True)  # 6. Añadimos esa fila al DataFrame resultado


mejores_por_alcaldia

## Metodología 

### Tercera Parte: Limpieza del DataSet

In [ ]:
# Verificar columnas clave antes de limpiar
print("Nulos por columna en el dataset original:")
print(df.isnull().sum())

# Crear una copia del DataFrame original para limpiarlo
df_limpio = df.copy()

# 🔹 1. Eliminar filas con NaN en columnas clave
columnas_clave = ['rating', 'userRatingCount', 'priceLevel', 'lat', 'lng']
df_limpio = df_limpio.dropna(subset=columnas_clave)

df_limpio

In [ ]:
# 🔹 2. Mapear priceLevel de texto a valores numéricos
price_mapping = {
    'PRICE_LEVEL_FREE': 0,
    'PRICE_LEVEL_INEXPENSIVE': 1,
    'PRICE_LEVEL_MODERATE': 2,
    'PRICE_LEVEL_EXPENSIVE': 3,
    'PRICE_LEVEL_VERY_EXPENSIVE': 4
}
df_limpio['priceLevel'] = df_limpio['priceLevel'].map(price_mapping)

# 🔹 3. Convertir columnas a formato numérico adecuado
df_limpio['rating'] = pd.to_numeric(df_limpio['rating'], errors='coerce')
df_limpio['userRatingCount'] = pd.to_numeric(df_limpio['userRatingCount'], errors='coerce')
df_limpio['priceLevel'] = pd.to_numeric(df_limpio['priceLevel'], errors='coerce').astype('Int64')
df_limpio['lat'] = pd.to_numeric(df_limpio['lat'], errors='coerce')
df_limpio['lng'] = pd.to_numeric(df_limpio['lng'], errors='coerce')

# 🔹 4. Eliminar duplicados
df_limpio = df_limpio.drop_duplicates()

# Reporte final
print("\nNulos por columna en el DataFrame limpio:")
print(df_limpio.isnull().sum())
print("\nTamaño final del DataFrame limpio:", df_limpio.shape)

In [ ]:
df_limpio

In [ ]:
#  1. Estadísticas generales
print("Estadísticas generales del DataFrame limpio:\n")
print(df_limpio.describe(include='all'))

# 2. Conteo total de taquerías
total_taquerias = len(df_limpio)
print(f"\nTotal de taquerías: {total_taquerias}")

#  3. Media y desviación estándar de calificación
media_rating = df_limpio['rating'].mean()
std_rating = df_limpio['rating'].std()
print(f"\nCalificación promedio: {media_rating:.2f}")
print(f"Desviación estándar de calificación: {std_rating:.2f}")

# 4. Media y desviación estándar del número de reseñas
media_resenas = df_limpio['userRatingCount'].mean()
std_resenas = df_limpio['userRatingCount'].std()
print(f"\n Reseñas promedio: {media_resenas:.2f}")
print(f"Desviación estándar de reseñas: {std_resenas:.2f}")




### Taquerías con peor y mejor calificación en DataFrame limpio

In [ ]:
df_limpio_ordenado = df_limpio.sort_values(by='rating', ascending=False)
df_limpio_ordenado.head(10)

In [ ]:
df_limpio_ordenado.tail(5)


## Metodología

### Cuarta parte : Ranking ponderado 

El objetivo es crear un ranking más justo y así encontrar las taquerías más recomendables tomando en cuenta:
Calidad (rating)
Popularidad (userRatingCount)
Accesibilidad (priceLevel)

Con el objetivo de determinar de forma objetiva cuáles son las taquerías más recomendables de la Ciudad de México, se diseñó un ranking ponderado que integra tres factores clave: la calidad percibida por los usuarios (rating), la popularidad basada en el número de reseñas (userRatingCount) y la accesibilidad económica (priceLevel).

#### Fundamento del Ranking 
* Fórmula 

$Ranking=\frac{rating x log(userRatingCoun)}{(priceLevel+1)}$

Variables utilizadas:
- rating: calificación promedio del establecimiento (escala de 1 a 5).
- userRatingCount: cantidad de reseñas emitidas por usuarios en Google Maps.
- priceLevel: nivel de precio del establecimiento (de 0 a 4, donde 0 = gratuito, 4 = muy caro).


Justificación 
- Se multiplica rating por el logaritmo natural de userRatingCount para favorecer lugares con buenas calificaciones respaldadas por muchas reseñas.
- El uso de log evita que valores extremadamente altos de reseñas dominen el cálculo.
- Se divide entre priceLevel + 1 para penalizar taquerías costosas y dar preferencia a las más accesibles. El +1 evita división entre cero

In [ ]:
# Asegurarse de eliminar cualquier fila con valores nulos en las columnas involucradas
df_ranking = df_limpio.dropna(subset=['rating', 'userRatingCount', 'priceLevel'])

# Eliminar entradas donde userRatingCount sea <= 0 para evitar log(0) o errores
df_ranking = df_ranking[df_ranking['userRatingCount'] > 0]

# Calcular el ranking ponderado
df_ranking['ranking'] = (df_ranking['rating'] * np.log(df_ranking['userRatingCount'])) / (df_ranking['priceLevel'] + 1)

print(df['priceLevel'].unique())
df_ranking

#### Ordenar en base a rankink ponderado

In [ ]:
# Ordenar de mayor a menor para obtener las mejores y peores taquerías
df_ranking.sort_values(by='ranking', ascending=False).head(10)





In [ ]:
df_ranking.sort_values(by='ranking', ascending=True).head(5)

In [ ]:
df_ranking[df_ranking['userRatingCount'] > 100].sort_values(by='ranking', ascending=True).head(10)

In [ ]:
#Lista de Alcaldias 
alcaldias_cdmx = [
    'Álvaro Obregón', 'Azcapotzalco', 'Benito Juárez', 'Coyoacán', 'Cuajimalpa',
    'Cuauhtémoc', 'Gustavo A. Madero', 'Iztacalco', 'Iztapalapa', 'La Magdalena Contreras',
    'Miguel Hidalgo', 'Milpa Alta', 'Tláhuac', 'Tlalpan', 'Venustiano Carranza', 'Xochimilco'
]

#Función para extraer alcaldías 
def extraer_alcaldia(direccion):
    for alcaldia in alcaldias_cdmx:
        if alcaldia.lower() in direccion.lower():
            return alcaldia
    return "No identificada"

df_copia_ranking = df_ranking.copy()
df_copia_ranking['alcaldia'] = df_copia_ranking['address'].apply(extraer_alcaldia)

df_copia_ranking

In [ ]:
# Obtener mejor taquería por alcaldía 
mejores_por_alcaldia_ranking = df_copia_ranking.loc[
    df_copia_ranking.groupby('alcaldia')['rating'].idxmax()
].reset_index(drop=True)

mejores_por_alcaldia_ranking

In [ ]:
columnas_clave2 = ['rating', 'userRatingCount']
df_limpio2 = df.dropna(subset=columnas_clave2)

df_limpio2['rating'] = pd.to_numeric(df_limpio2['rating'], errors='coerce')
df_limpio2['userRatingCount'] = pd.to_numeric(df_limpio2['userRatingCount'], errors='coerce')

df_limpio2 = df_limpio2.drop_duplicates()


df_limpio2




In [ ]:
df_limpio2 = df_limpio2[df_limpio2['userRatingCount'] > 0]

df_limpio2['ranking2'] = (df_ranking['rating'] * np.log(df_ranking['userRatingCount'])) 

df_limpio2

In [ ]:
df_limpio2.sort_values(by='ranking2', ascending=False).head(10)

In [ ]:

# Centrar el mapa en CDMX
mapa = folium.Map(location=[19.4326, -99.1332], zoom_start=12)

# Agregar marcadores al mapa desde df_limpio
for index, row in df_limpio.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=f"{row['name']}<br>Rating: {row['rating']}<br>Precio: {row['priceLevel']}",
        tooltip=row['name'],
        icon=folium.Icon(color="red", icon="cutlery", prefix="fa")
    ).add_to(mapa)

# Mostrar el mapa
mapa